In [1]:
from keras.utils import to_categorical
import pandas as pd
import numpy as np
import cv2

def preprocessing(data, labels, val=True):
    
    data = pd.read_csv(data)
    labels = pd.read_csv(labels)

    if(val):
        ## ---------------- Data preparation ---------------- ##
        X_train = []
        for i in range(data.shape[0]):
            img = np.uint8(data.iloc[i])
            edited = cv2.Canny(img, 10, 30)
            edited = cv2.GaussianBlur(edited, (5, 5), 0)
            X_train.append(edited.reshape((1,-1))[0])

        data = pd.DataFrame(X_train)
        ## -------------------------------------------------- ##

    return data, labels

Using TensorFlow backend.


In [2]:
IMG_SIZE = 48

X_train, y_train = preprocessing(data='data/x_train_gr_smpl.csv', labels='data/y_train_smpl.csv')
X_train['label'] = y_train
X_train = X_train.sample(frac=1)
y_train = X_train['label']
X_train = X_train.iloc[:, :-1]
y_train = to_categorical(y_train)
X_train = np.array(X_train).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
X_train = X_train/255.0

X_test, y_test = preprocessing(data='data/x_test_gr_smpl.csv', labels='data/y_test_smpl.csv')
X_test['label'] = y_test
X_test = X_test.sample(frac=1)
y_test = X_test['label']
X_test = X_test.iloc[:, :-1]
y_test = to_categorical(y_test)
X_test = np.array(X_test).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
X_test = X_test/255.0

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
import time


gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.7)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

conv_layers = [1, 2, 3]
feature_maps = [5, 20]
win_sizes = [3, 5]
dense_layers = [0, 1, 2]
layer_sizes = [64, 128, 256]

for conv_layer in conv_layers:
    for feature_map in feature_maps:
        for win_size in win_sizes:
            for dense_layer in dense_layers:
                for layer_size in layer_sizes:
                

                    NAME = f"{conv_layer}ConvL-{feature_map}FMap-{win_size}WinSize-{dense_layer}DenseL-{layer_size}Neurons-{int(time.time())}"
                    tensorboard = TensorBoard(log_dir=f"logs\{NAME}")
                    print("----------------------------------------")
                    print(NAME)

                    model = Sequential()

                    model.add(Conv2D(feature_map, (win_size, win_size), input_shape=X_train.shape[1:], activation='relu'))
                    model.add(MaxPooling2D(pool_size=(2, 2)))

                    for _ in range(conv_layer-1):
                        model.add(Conv2D(feature_map, (win_size, win_size), activation='relu'))
                        model.add(MaxPooling2D(pool_size=(2, 2)))

                    model.add(Flatten())
                    for _ in range(dense_layer):
                        model.add(Dense(layer_size, activation='relu'))
                        model.add(Dropout(0.2))

                    model.add(Dense(10, activation='softmax'))

                    model.compile(loss="categorical_crossentropy",
                                 optimizer="adam",
                                 metrics=["accuracy"])

                    model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=200, epochs=10, callbacks=[tensorboard])
                    model.save(f"models\{NAME}.model")

----------------------------------------
1ConvL-5FMap-3WinSize-0DenseL-64Neurons-1574959257
Instructions for updating:
Colocations handled automatically by placer.
Train on 12660 samples, validate on 4170 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
12660/12660 [==============================] - 3s 233us/sample - loss: 1.3376 - acc: 0.6474 - val_loss: 0.7515 - val_acc: 0.7736
Epoch 2/10
12660/12660 [==============================] - 1s 96us/sample - loss: 0.5073 - acc: 0.8454 - val_loss: 0.4967 - val_acc: 0.8228
Epoch 3/10
12660/12660 [==============================] - 1s 96us/sample - loss: 0.3339 - acc: 0.8945 - val_loss: 0.4213 - val_acc: 0.8540
Epoch 4/10
12660/12660 [==============================] - 1s 97us/sample - loss: 0.2634 - acc: 0.9156 - val_loss: 0.3792 - val_acc: 0.8640
Epoch 5/10
12660/12660 [==============================] - 1s 106us/sample - loss: 0.2220 - acc: 0.9301 - val_loss: 0.3497 - val_acc: 0.8729
Epoch 6/10
12660/12660 [==================

12660/12660 [==============================] - 1s 97us/sample - loss: 0.1302 - acc: 0.9598 - val_loss: 0.3158 - val_acc: 0.8868
----------------------------------------
1ConvL-20FMap-3WinSize-0DenseL-256Neurons-1574959332
Train on 12660 samples, validate on 4170 samples
Epoch 1/10
12660/12660 [==============================] - 2s 136us/sample - loss: 0.9879 - acc: 0.7195 - val_loss: 0.5376 - val_acc: 0.7890
Epoch 2/10
12660/12660 [==============================] - 1s 117us/sample - loss: 0.3228 - acc: 0.8921 - val_loss: 0.4056 - val_acc: 0.8597
Epoch 3/10
12660/12660 [==============================] - 1s 118us/sample - loss: 0.2288 - acc: 0.9234 - val_loss: 0.3449 - val_acc: 0.8818
Epoch 4/10
12660/12660 [==============================] - 1s 117us/sample - loss: 0.1797 - acc: 0.9404 - val_loss: 0.3254 - val_acc: 0.8866
Epoch 5/10
12660/12660 [==============================] - 2s 120us/sample - loss: 0.1514 - acc: 0.9493 - val_loss: 0.3209 - val_acc: 0.8916
Epoch 6/10
12660/12660 [=====

12660/12660 [==============================] - 1s 103us/sample - loss: 0.4364 - acc: 0.8670 - val_loss: 0.4607 - val_acc: 0.8355
Epoch 3/10
12660/12660 [==============================] - 1s 101us/sample - loss: 0.2845 - acc: 0.9102 - val_loss: 0.3908 - val_acc: 0.8674
Epoch 4/10
12660/12660 [==============================] - 1s 100us/sample - loss: 0.2217 - acc: 0.9298 - val_loss: 0.3561 - val_acc: 0.8801
Epoch 5/10
12660/12660 [==============================] - 1s 100us/sample - loss: 0.1921 - acc: 0.9367 - val_loss: 0.3499 - val_acc: 0.8803
Epoch 6/10
12660/12660 [==============================] - 1s 102us/sample - loss: 0.1704 - acc: 0.9430 - val_loss: 0.3228 - val_acc: 0.8911
Epoch 7/10
12660/12660 [==============================] - 1s 102us/sample - loss: 0.1542 - acc: 0.9478 - val_loss: 0.3389 - val_acc: 0.8839
Epoch 8/10
12660/12660 [==============================] - 1s 104us/sample - loss: 0.1425 - acc: 0.9505 - val_loss: 0.3243 - val_acc: 0.8916
Epoch 9/10
12660/12660 [=======

12660/12660 [==============================] - 2s 129us/sample - loss: 0.1626 - acc: 0.9460 - val_loss: 0.2796 - val_acc: 0.9048
Epoch 4/10
12660/12660 [==============================] - 2s 129us/sample - loss: 0.1344 - acc: 0.9557 - val_loss: 0.3058 - val_acc: 0.8947
Epoch 5/10
12660/12660 [==============================] - 2s 130us/sample - loss: 0.0883 - acc: 0.9728 - val_loss: 0.2823 - val_acc: 0.9086
Epoch 6/10
12660/12660 [==============================] - 2s 127us/sample - loss: 0.0784 - acc: 0.9744 - val_loss: 0.2486 - val_acc: 0.9213
Epoch 7/10
12660/12660 [==============================] - 2s 126us/sample - loss: 0.0550 - acc: 0.9839 - val_loss: 0.2475 - val_acc: 0.9247
Epoch 8/10
12660/12660 [==============================] - 2s 127us/sample - loss: 0.0460 - acc: 0.9862 - val_loss: 0.2556 - val_acc: 0.9132
Epoch 9/10
12660/12660 [==============================] - 2s 129us/sample - loss: 0.0413 - acc: 0.9884 - val_loss: 0.2199 - val_acc: 0.9285
Epoch 10/10
12660/12660 [======

12660/12660 [==============================] - 1s 109us/sample - loss: 0.1534 - acc: 0.9472 - val_loss: 0.3353 - val_acc: 0.8880
Epoch 7/10
12660/12660 [==============================] - 1s 112us/sample - loss: 0.1471 - acc: 0.9476 - val_loss: 0.3445 - val_acc: 0.8794
Epoch 8/10
12660/12660 [==============================] - 1s 110us/sample - loss: 0.1211 - acc: 0.9577 - val_loss: 0.3355 - val_acc: 0.8904
Epoch 9/10
12660/12660 [==============================] - 1s 110us/sample - loss: 0.1104 - acc: 0.9620 - val_loss: 0.3198 - val_acc: 0.8921
Epoch 10/10
12660/12660 [==============================] - 1s 110us/sample - loss: 0.1026 - acc: 0.9645 - val_loss: 0.3322 - val_acc: 0.8942
----------------------------------------
1ConvL-20FMap-5WinSize-1DenseL-128Neurons-1574959628
Train on 12660 samples, validate on 4170 samples
Epoch 1/10
12660/12660 [==============================] - 3s 201us/sample - loss: 0.7840 - acc: 0.7348 - val_loss: 0.4109 - val_acc: 0.8590
Epoch 2/10
12660/12660 [===

...

12660/12660 [==============================] - 2s 186us/sample - loss: 0.1222 - acc: 0.9580 - val_loss: 0.3027 - val_acc: 0.9048
----------------------------------------
3ConvL-5FMap-3WinSize-1DenseL-64Neurons-1574962100
Train on 12660 samples, validate on 4170 samples
Epoch 1/10
12660/12660 [==============================] - 4s 327us/sample - loss: 2.1285 - acc: 0.2622 - val_loss: 1.8183 - val_acc: 0.5269
Epoch 2/10
12660/12660 [==============================] - 2s 131us/sample - loss: 1.2610 - acc: 0.6018 - val_loss: 0.7522 - val_acc: 0.7396
Epoch 3/10
12660/12660 [==============================] - 2s 134us/sample - loss: 0.7254 - acc: 0.7308 - val_loss: 0.5770 - val_acc: 0.7878
Epoch 4/10
12660/12660 [==============================] - 2s 133us/sample - loss: 0.5796 - acc: 0.7818 - val_loss: 0.5004 - val_acc: 0.8141
Epoch 5/10
12660/12660 [==============================] - 2s 134us/sample - loss: 0.5008 - acc: 0.8025 - val_loss: 0.4826 - val_acc: 0.8211
Epoch 6/10
12660/12660 [======

12660/12660 [==============================] - 2s 161us/sample - loss: 0.3946 - acc: 0.8491 - val_loss: 0.3928 - val_acc: 0.8664
Epoch 3/10
12660/12660 [==============================] - 2s 161us/sample - loss: 0.2454 - acc: 0.9089 - val_loss: 0.3043 - val_acc: 0.8892
Epoch 4/10
12660/12660 [==============================] - 2s 161us/sample - loss: 0.1817 - acc: 0.9299 - val_loss: 0.3126 - val_acc: 0.8904
Epoch 5/10
12660/12660 [==============================] - 3s 201us/sample - loss: 0.1369 - acc: 0.9503 - val_loss: 0.2956 - val_acc: 0.8954
Epoch 6/10
12660/12660 [==============================] - 2s 162us/sample - loss: 0.1174 - acc: 0.9551 - val_loss: 0.2612 - val_acc: 0.9089
Epoch 7/10
12660/12660 [==============================] - 2s 163us/sample - loss: 0.1057 - acc: 0.9603 - val_loss: 0.2357 - val_acc: 0.9132
Epoch 8/10
12660/12660 [==============================] - 2s 164us/sample - loss: 0.0833 - acc: 0.9684 - val_loss: 0.2626 - val_acc: 0.9082
Epoch 9/10
12660/12660 [=======

12660/12660 [==============================] - 2s 140us/sample - loss: 0.3498 - acc: 0.8729 - val_loss: 0.4091 - val_acc: 0.8499
Epoch 6/10
12660/12660 [==============================] - 2s 141us/sample - loss: 0.3089 - acc: 0.8897 - val_loss: 0.3995 - val_acc: 0.8588
Epoch 7/10
12660/12660 [==============================] - 2s 143us/sample - loss: 0.2799 - acc: 0.9013 - val_loss: 0.4033 - val_acc: 0.8588
Epoch 8/10
12660/12660 [==============================] - 2s 143us/sample - loss: 0.2594 - acc: 0.9063 - val_loss: 0.3701 - val_acc: 0.8679
Epoch 9/10
12660/12660 [==============================] - 2s 142us/sample - loss: 0.2383 - acc: 0.9122 - val_loss: 0.3666 - val_acc: 0.8667
Epoch 10/10
12660/12660 [==============================] - 2s 145us/sample - loss: 0.2226 - acc: 0.9179 - val_loss: 0.3621 - val_acc: 0.8688
----------------------------------------
3ConvL-20FMap-5WinSize-1DenseL-256Neurons-1574962755
Train on 12660 samples, validate on 4170 samples
Epoch 1/10
12660/12660 [===

12660/12660 [==============================] - 3s 218us/sample - loss: 0.1198 - acc: 0.9548 - val_loss: 0.2446 - val_acc: 0.9170
Epoch 9/10
12660/12660 [==============================] - 2s 167us/sample - loss: 0.1055 - acc: 0.9603 - val_loss: 0.2671 - val_acc: 0.9139
Epoch 10/10
12660/12660 [==============================] - 2s 180us/sample - loss: 0.0901 - acc: 0.9662 - val_loss: 0.2568 - val_acc: 0.9180
----------------------------------------
3ConvL-5FMap-3WinSize-2DenseL-256Neurons-1574963104
Train on 12660 samples, validate on 4170 samples
Epoch 1/10
12660/12660 [==============================] - 5s 389us/sample - loss: 1.4247 - acc: 0.5258 - val_loss: 0.6393 - val_acc: 0.7746
Epoch 2/10
12660/12660 [==============================] - 2s 140us/sample - loss: 0.5381 - acc: 0.7933 - val_loss: 0.5256 - val_acc: 0.8201
Epoch 3/10
12660/12660 [==============================] - 3s 232us/sample - loss: 0.3830 - acc: 0.8528 - val_loss: 0.3824 - val_acc: 0.8729
Epoch 4/10
12660/12660 [====

Train on 12660 samples, validate on 4170 samples
Epoch 1/10
12660/12660 [==============================] - 5s 430us/sample - loss: 1.3528 - acc: 0.5386 - val_loss: 0.6037 - val_acc: 0.7871
Epoch 2/10
12660/12660 [==============================] - 2s 170us/sample - loss: 0.5177 - acc: 0.7994 - val_loss: 0.4319 - val_acc: 0.8230
Epoch 3/10
12660/12660 [==============================] - 2s 173us/sample - loss: 0.3433 - acc: 0.8634 - val_loss: 0.3604 - val_acc: 0.8801
Epoch 4/10
12660/12660 [==============================] - 2s 170us/sample - loss: 0.2400 - acc: 0.9095 - val_loss: 0.2938 - val_acc: 0.8918
Epoch 5/10
12660/12660 [==============================] - 2s 173us/sample - loss: 0.1864 - acc: 0.9295 - val_loss: 0.2580 - val_acc: 0.9022
Epoch 6/10
12660/12660 [==============================] - 2s 174us/sample - loss: 0.1586 - acc: 0.9419 - val_loss: 0.2408 - val_acc: 0.9065
Epoch 7/10
12660/12660 [==============================] - 2s 175us/sample - loss: 0.1342 - acc: 0.9484 - val_lo

tensorboard --logdir=logs/ --host localhost --port 8088